In [1]:
"""Tutorial on how to create an autoencoder w/ Tensorflow.
Parag K. Mital, Jan 2016
"""
# %% Imports
import tensorflow as tf
import numpy as np
import math


# %% Autoencoder definition
def autoencoder(dimensions=[784, 512, 256, 64]):
    """
    Build a deep autoencoder w/ tied weights. w = wT
    return a dict.
    
    Parameters
    ----------
    dimensions : list, optional
        The number of neurons for each layer of the autoencoder.
    Returns
    -------
    x : Tensor
        Input placeholder to the network
    z : Tensor
        Inner-most latent representation
    y : Tensor
        Output reconstruction of the input
    cost : Tensor
        Overall cost to use for training
    """
    # %% input to the network
    x = tf.placeholder(tf.float32, [None, dimensions[0]], name='x')
    current_input = x
    print ('current_input', current_input.shape)
    # %% Build the encoder
    encoder = [] # for putting the weight of encoder, w1,w2,..
    for layer_i, n_output in enumerate(dimensions[1:]):
        print ('layer_i-encoder', layer_i)
        print ('n_output-encoder', n_output)
        n_input = int(current_input.get_shape()[1]) # [0]: batch_szie, [1]:input_dim
        print ('n_input', n_input)
        W = tf.Variable(
            tf.random_uniform([n_input, n_output],
                              minval = -1.0 / math.sqrt(n_input),
                              maxval = 1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        # saving layer of encoding for decoder
        encoder.append(W)
        output = tf.nn.tanh(tf.matmul(current_input, W) + b)
        # assign current_input
        current_input = output

    # %% latent representation (output of encoder)
    z = current_input
    encoder.reverse() # [...,w2,w1]

    # %% Build the decoder using the same weights
    for layer_i, n_output in enumerate(dimensions[:-1][::-1]):
        print ('layer_i-decoder', layer_i)
        print ('n_output-decoder', n_output)
        W = tf.transpose(encoder[layer_i])
        b = tf.Variable(tf.zeros([n_output]))
        output = tf.nn.tanh(tf.matmul(current_input, W) + b)
        # assign current_input
        current_input = output

    # %% now have the reconstruction through the network
    y = current_input

    # %% cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x)) # constrution loss
    return {'x': x, 'z': z, 'y': y, 'cost': cost}


# %% Basic test
def test_mnist():
    """Test the autoencoder using MNIST."""
    import tensorflow as tf
    import tensorflow.examples.tutorials.mnist.input_data as input_data
    import matplotlib.pyplot as plt

    # %%
    # load MNIST as before
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    # why?
    mean_img = np.mean(mnist.train.images, axis=0)
    print ('mean_img', mean_img.shape)
    ae = autoencoder(dimensions=[784, 256, 64])

    # %%
    learning_rate = 0.001
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(ae['cost']) # ae['cost']: tf.placerholder

    # %%
    # We create a session to use the graph
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # %%
    # Fit all training data
    batch_size = 50
    n_epochs = 10
    for epoch_i in range(n_epochs):
        print ('num_samples : ', mnist.train.num_examples)
        for batch_i in range(mnist.train.num_examples // batch_size):
            batch_xs, _ = mnist.train.next_batch(batch_size) # (50, 784)
            #print ('batch_xs', batch_xs.shape)
            # the below like normalization
            train = np.array([img - mean_img for img in batch_xs])
            #print ('train', train.shape)
            # train
            sess.run(optimizer, feed_dict={ae['x']: train})
        print ('epoch_i', epoch_i)
        print('cost', sess.run(ae['cost'], feed_dict={ae['x']: train})) # feed_dict = {tf.placeholder: real_data}

    # %%
    # Plot example reconstructions
    n_examples = 15
    test_xs, _ = mnist.test.next_batch(n_examples)
    test_xs_norm = np.array([img - mean_img for img in test_xs])
    # resconstruct --->y
    recon = sess.run(ae['y'], feed_dict={ae['x']: test_xs_norm})
    print ('recon',recon)
    #fig, axs = plt.subplots(2, n_examples, figsize=(10, 2))
    print ('n_examples', n_examples.shape)
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            np.reshape(test_xs[example_i, :], (28, 28)))
        axs[1][example_i].imshow(
            np.reshape([recon[example_i, :] + mean_img], (28, 28)))
#     fig.show()
#     plt.draw()
#     plt.waitforbuttonpress()




/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# %%
if __name__ == '__main__':
    test_mnist()
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
mean_img (784,)
current_input (?, 784)
layer_i-encoder 0
n_output-encoder 256
n_input 784
layer_i-encoder 1
n_output-encoder 64
n_input 256
layer_i-decoder 0
n_output-decoder 256
layer_i-decoder 1
n_output-decoder 784
num_samples :  55000
epoch_i 0
cost 451.73486
num_samples :  55000
epoch_i 1
cost 428.62915
num_samples :  55000
epoch_i 2
cost 416.38776
num_samples :  55000
epoch_i 3
cost 430.29376
num_samples :  55000
epoch_i 4
cost 433.11243
num_samples :  55000
epoch_i 5
cost 382.50592
num_samples :  55000
epoch_i 6
cost 423.29608
num_samples :  55000
epoch_i 7
cost 391.61313
num_samples :  55000
epoch_i 8
cost 390.31744
num_samples :  55000
epoch_i 9
cost 374.72513
recon [[ 3.7915152e-03  3.0337158e-04 -5.7011534e-04 ... -3.9455949e-04
  -2.2623525e-03 -2.9706312e-03]
 [ 5.4189313e-04  3.496

AttributeError: 'int' object has no attribute 'shape'

# tf.random_uniform(shape, minval=0, maxval=None) : Outputs random values from a uniform distribution.

In [ ]:
c =  tf.random_uniform([1,2],minval = -1, maxval = 1)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
c = sess.run(c)
print (c, c.shape)

In [ ]:
t = [1, 2]
t.reverse()
t

In [ ]:
dimensions=[784, 256, 64]
dimensions[:-1][::-1]